In [4]:
import pandas as pd

In [5]:
# Read in data from crew csv
crew = pd.read_csv('../data/raw/title.crew.tsv', sep='\t')

In [6]:
crew

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N
...,...,...,...
9275709,tt9916848,"nm5519375,nm5519454","nm6182221,nm1628284,nm2921377"
9275710,tt9916850,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
9275711,tt9916852,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
9275712,tt9916856,nm10538645,nm6951431
